Touching the first 100 papers under CS topic of ArXiv<br>
• https://arxiv.org/list/cs/recent<br>
• Collecting the title, Subjects, Authors, and abstract<br>
• Save all into a csv files.<br>
• Note: we need to collect the abstract from the detailed page

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from zenrows import ZenRowsClient

base_url = "https://arxiv.org/list/cs/pastweek?skip=0&show=100"
client = ZenRowsClient("14d2f2062a095a7caa05e0142ce147f5174b0a7e")
response = client.get(base_url)
# print(response.text)

In [3]:
base_url = "https://arxiv.org/list/cs/pastweek?skip=0&show=100"
headers = {
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}
# response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
data = []
papers = soup.find_all('dd')
for paper in papers:
    title = paper.find('div', class_='list-title mathjax').text.replace('Title:', '').strip()
    subjects = paper.find('div', class_='list-subjects').text.replace('Subjects:', '').strip()
    # authors = paper.find('div', class_='list-authors').text.replace('Authors:', '').strip()
    authors = paper.find('div', class_='list-authors').text.replace("\n", "").replace("Authors:", "").strip()
    
    data.append({
        'Title': title,
        'Subjects': subjects,
        'Authors': authors,
    })

In [5]:
import time
import random
headers_new = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
links = soup.find_all('dt')
abstracts = []
for url in links:
    paper_url = "https://arxiv.org" + url.find('a', href=True)['href']
    # paper_response = requests.get(paper_url, headers=headers_new)
    paper_response = client.get(paper_url)
    
    # time.sleep(random.uniform(0.8, 2)) 
    paper_soup = BeautifulSoup(paper_response.text, 'html.parser')
    # print(paper_soup)
    
    abstract = paper_soup.find('blockquote', class_='abstract').text.replace("Abstract:", "").strip()
    
    abstracts.append({
        'Abstract': abstract
    })

In [6]:
abstracts_df = pd.DataFrame(abstracts)
data_df = pd.DataFrame(data)

In [7]:
merged_df = pd.concat([data_df, abstracts_df], axis=1)
merged_df.head()

,Title,Subjects,Authors,Abstract
0,Point Could Mamba: Point Cloud Learning via St...,Computer Vision and Pattern Recognition (cs.CV),"Tao Zhang, Xiangtai Li, Haobo Yuan, Shunping J...","In this work, for the first time, we demonstra..."
1,Mitigating Reversal Curse via Semantic-aware P...,Computation and Language (cs.CL); Artificial I...,"Qingyan Guo, Rui Wang, Junliang Guo, Xu Tan, J...",While large language models (LLMs) have achiev...
2,An Experimental Study of Low-Latency Video Str...,Multimedia (cs.MM); Performance (cs.PF),"Imran Khan, Tuyen X. Tran, Matti Hiltunen, The...",Low-latency video streaming over 5G has become...
3,AtP*: An efficient and scalable method for loc...,Machine Learning (cs.LG); Computation and Lang...,"János Kramár, Tom Lieberum, Rohin Shah, Neel N...",Activation Patching is a method of directly co...
4,Neural Acceleration of Incomplete Cholesky Pre...,"Distributed, Parallel, and Cluster Computing (...","Joshua Dennis Booth, Hongyang Sun, Trevor Garnett",The solution of a sparse system of linear equa...


In [8]:
merged_df.to_csv('merged_data.csv', index=False)